# 对话代理 Conversational agent

 - [一、设置OpenAI API Key](#一、设置OpenAI-API-Key)
 - [二、逐步构建Agent](#二、逐步构建Agent)
     - [2.1 定义Tools和Function](##2.1-定义Tools和Function)
     - [2.2 加入中间步骤结果](##2.2-加入中间步骤结果)
     - [2.3 加入对话历史](##2.3-加入对话历史)
 - [三、创建对话机器人](#三、创建对话机器人)
 - [四、英文版提示](#四、英文版提示)

# 一、设置OpenAI-API-Key

详细内容见`设置OpenAI_API_KEY.ipynb`文件

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain_openai.chat_models import ChatOpenAI

loaded = load_dotenv(find_dotenv(), override=True)
# 从环境变量中获取 OpenAI API Key 或者直接赋值
API_KEY = os.getenv("API_KEY")

# 如果您使用的是官方 API，就直接用 https://api.siliconflow.cn/v1 就行。
BASE_URL = "https://api.siliconflow.cn/v1"

# 二、逐步构建Agent

## 2.1 定义Tools和Function

In [4]:
# 导入tool包和所需的库
from langchain.tools import tool
import requests
from pydantic import BaseModel, Field
import datetime

In [5]:
# 定义输入格式
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="要获取天气数据的位置的纬度") 
    longitude: float = Field(..., description="要获取天气数据的位置的经度") 

# 使用 @tool 装饰器并指定输入格式
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """"根据给定的坐标位置获得温度"""
    
    # Open Meteo API 的URL
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # 请求参数
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # 发送 API 请求
    response = requests.get(BASE_URL, params=params)
    
    # 检查响应状态码
    if response.status_code == 200:
        # 解析 JSON 响应
        results = response.json()
    else:
        # 处理请求失败的情况
        raise Exception(f"API Request failed with status code: {response.status_code}")

    # 获取当前 UTC 时间
    current_utc_time = datetime.datetime.now(datetime.UTC)
    
    # 将时间字符串转换为 datetime 对象
    time_list = [datetime.datetime.fromisoformat(time_str).replace(tzinfo=datetime.timezone.utc) for time_str in results['hourly']['time']]
    
    # 获取温度列表
    temperature_list = results['hourly']['temperature_2m']
    
    # 找到最接近当前时间的索引
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    
    # 获取当前温度
    current_temperature = temperature_list[closest_time_index]
    
    # 返回当前温度的字符串形式
    return f'现在温度是 {current_temperature}°C。'


In [6]:
import wikipedia

# 定义维基百科搜索的tool
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]: #取前三个页面标题
        try:
            #使用 wikipedia 模块的 page 函数，获取指定标题的维基百科页面对象。
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False) 
            # 获取页面摘要
            summaries.append(f"页面: {page_title}\n摘要: {wiki_page.summary}")
        except (
            wikipedia.exceptions.PageError,
            wikipedia.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "维基百科没有搜索到合适的结果"
    return "\n\n".join(summaries)

In [7]:
# 加入tools列表
tools = [get_current_temperature, search_wikipedia]

In [8]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers import ToolsAgentOutputParser

In [9]:
# 将工具格式化为 OpenAI 函数
functions = [convert_to_openai_function(f) for f in tools]

tools = [{"type": "function", "function": x} for x in functions]

# 创建 ChatOpenAI 模型，设置温度为 0，绑定生成的 OpenAI 函数列表

model = ChatOpenAI(temperature=0, model_name="deepseek-ai/DeepSeek-V3", max_tokens=4096,
                        openai_api_key=API_KEY, openai_api_base=BASE_URL, max_retries=3,
                        seed=42, presence_penalty=0.1, frequency_penalty=0.1,
                        extra_body={
                            "enable_thinking": False
                        }
                        ).bind(tools=tools)

# 创建 ChatPromptTemplate，从消息模板中获取用户输入
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

# 创建处理链，将 prompt、model 和 OpenAIFunctionsAgentOutputParser 连接起来
chain = prompt | model | ToolsAgentOutputParser()

In [10]:
# 调用
result = chain.invoke({"input": "现在圣佛朗西斯科的温度是多少?"})

In [11]:
# 查看调用的工具
result[0].tool

'get_current_temperature'

In [12]:
# 查看工具的输入
result[0].tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

## 2.2 加入中间步骤结果

In [13]:
# 创建 ChatPromptTemplate，从消息模板中获取用户输入
from langchain.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个安全且乐于助人的助手"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad") # 解析和验证关键字参数中的输入数据
])

In [14]:
# 创建处理链，将 prompt、model 和 OpenAIFunctionsAgentOutputParser 连接起来
chain = prompt | model | ToolsAgentOutputParser()

In [15]:
# 调用
result1 = chain.invoke({
    "input": "现在圣佛朗西斯科的温度是多少?",
    "agent_scratchpad": []
})

In [16]:
# 查看调用的工具
result1[0].tool

'get_current_temperature'

In [18]:
# 获取tool调用的结果
observation = get_current_temperature.invoke(result1[0].tool_input)

In [19]:
# 查看结果
observation

'现在温度是 11.6°C'

In [21]:
# 查看输出类型
type(result1[0])

langchain.agents.output_parsers.tools.ToolAgentAction

In [31]:
from langchain.agents.format_scratchpad import format_to_tool_messages

In [24]:
# 查看调用日志
result1[0].message_log

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '019763d3374456f690cd602e78209fd7', 'function': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 211, 'total_tokens': 229, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '019763d32febb376d82cfcc8e1e23a59', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--59132dc6-8cd8-4bd8-bca2-a52e91d9c352-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 37.7749, 'longitude': -122.4194}, 'id': '019763d3374456f690cd602e78209fd7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 211, 'output_tokens': 18, 'total_tokens': 229, 'input_token_details': {}, 'output_token_details': {}})]

In [32]:
# 将Function和Tool的结果转换成openai函数格式
format_to_tool_messages([(result1[0], observation), ])

[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '019763d3374456f690cd602e78209fd7', 'function': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 211, 'total_tokens': 229, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'deepseek-ai/DeepSeek-V3', 'system_fingerprint': '', 'id': '019763d32febb376d82cfcc8e1e23a59', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--59132dc6-8cd8-4bd8-bca2-a52e91d9c352-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 37.7749, 'longitude': -122.4194}, 'id': '019763d3374456f690cd602e78209fd7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 211, 'output_tokens': 18, 'total_tokens': 229, 'input_token_details': {}, 'output_token_details': {}}),
 ToolMessage(content='现在温度是 11.6°C'

In [29]:
# 构造成Agent的输入格式，agent_scratchpad是中间步骤的结果
result2 = chain.invoke({
    "input": "现在圣佛朗西斯科的温度是多少?", 
    "agent_scratchpad": format_to_tool_messages([(result1[0], observation)])
})

In [30]:
# 查看Agent的输出结果
result2

AgentFinish(return_values={'output': '现在圣佛朗西斯科的温度是 11.6°C。'}, log='现在圣佛朗西斯科的温度是 11.6°C。')

In [41]:
from langchain.schema.agent import AgentFinish

def run_agent(user_input):
    # 存储中间步骤的列表
    intermediate_steps = []

    # 循环执行 agent 操作
    while True:
        # 调用处理链的 invoke 方法，传递用户输入和中间步骤的 OpenAI 函数形式
        result = chain.invoke({
            "input": user_input,
            "agent_scratchpad": format_to_tool_messages(intermediate_steps)
        })

        # 如果结果是 AgentFinish 类型，则结束运行并返回结果
        if isinstance(result, AgentFinish):
            return result

        # 根据结果中的工具名称选择相应的工具函数
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result[0].tool]

        # 运行选定的工具函数，并获取观察结果
        observation = tool.run(result[0].tool_input)

        # 将结果和观察结果添加到中间步骤列表
        intermediate_steps.append((result[0], observation))


In [42]:
from langchain.schema.runnable import RunnablePassthrough

# 创建 RunnablePassthrough，用于将 agent_scratchpad存储的中间步骤结果转换为 OpenAI 函数形式
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad=lambda x: format_to_tool_messages(x["intermediate_steps"])
) | chain

In [43]:
# 定义一个运行 agent 的函数
def run_agent(user_input):
    # 存储中间步骤的列表
    intermediate_steps = []

    # 循环执行 agent 操作
    while True:
        # 通过 agent_chain 调用 invoke 方法，传递用户输入和中间步骤的列表
        result = agent_chain.invoke({
            "input": user_input,
            "intermediate_steps": intermediate_steps
        })

        # 如果结果是 AgentFinish 类型，则结束循环并返回结果
        if isinstance(result, AgentFinish):
            return result

        # 根据结果中的工具名称选择相应的工具函数
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result[0].tool]

        # 运行选定的工具函数，并获取观察结果
        observation = tool.run(result[0].tool_input)

        # 将结果和观察结果添加到中间步骤列表
        intermediate_steps.append((result[0], observation))


In [44]:
run_agent("圣弗朗西斯科的温度是多少？")

AgentFinish(return_values={'output': '圣弗朗西斯科（旧金山）的当前温度是 11.6°C。'}, log='圣弗朗西斯科（旧金山）的当前温度是 11.6°C。')

In [45]:
run_agent("什么是langchain?")

AgentFinish(return_values={'output': 'LangChain 是一个软件框架，旨在帮助将大型语言模型（LLMs）集成到应用程序中。它的主要用途与语言模型的通用用途类似，包括文档分析与摘要、聊天机器人以及代码分析等。\n\n### 关键点：\n1. **功能**：\n   - 提供工具和接口，简化语言模型与应用程序的集成。\n   - 支持多种用例，如文档处理、对话系统和代码生成。\n\n2. **优势**：\n   - 通过框架化的方式，开发者可以更高效地利用语言模型的能力。\n   - 可能支持检索增强生成（RAG）等技术，使模型能够动态引用外部数据。\n\n3. **应用场景**：\n   - 聊天机器人：结合内部或权威数据生成更准确的回答。\n   - 文档分析：快速提取和总结文档内容。\n   - 代码分析：辅助开发人员理解或生成代码。\n\n如果你对 LangChain 的具体技术细节或实现方式感兴趣，可以进一步探讨！'}, log='LangChain 是一个软件框架，旨在帮助将大型语言模型（LLMs）集成到应用程序中。它的主要用途与语言模型的通用用途类似，包括文档分析与摘要、聊天机器人以及代码分析等。\n\n### 关键点：\n1. **功能**：\n   - 提供工具和接口，简化语言模型与应用程序的集成。\n   - 支持多种用例，如文档处理、对话系统和代码生成。\n\n2. **优势**：\n   - 通过框架化的方式，开发者可以更高效地利用语言模型的能力。\n   - 可能支持检索增强生成（RAG）等技术，使模型能够动态引用外部数据。\n\n3. **应用场景**：\n   - 聊天机器人：结合内部或权威数据生成更准确的回答。\n   - 文档分析：快速提取和总结文档内容。\n   - 代码分析：辅助开发人员理解或生成代码。\n\n如果你对 LangChain 的具体技术细节或实现方式感兴趣，可以进一步探讨！')

In [46]:
run_agent("你好！")

AgentFinish(return_values={'output': '你好！有什么可以帮您的吗？'}, log='你好！有什么可以帮您的吗？')

In [49]:
tools

[{'type': 'function',
  'function': {'name': 'get_current_temperature',
   'description': '"根据给定的坐标位置获得温度',
   'parameters': {'properties': {'latitude': {'description': '要获取天气数据的位置的纬度',
      'type': 'number'},
     'longitude': {'description': '要获取天气数据的位置的经度', 'type': 'number'}},
    'required': ['latitude', 'longitude'],
    'type': 'object'}}},
 {'type': 'function',
  'function': {'name': 'search_wikipedia',
   'description': 'Run Wikipedia search and get page summaries.',
   'parameters': {'properties': {'query': {'type': 'string'}},
    'required': ['query'],
    'type': 'object'}}}]

In [50]:
# 使用AgentExecutor对agent进行封装
from langchain.agents import AgentExecutor

tools = [get_current_temperature, search_wikipedia]

agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [51]:
agent_executor.invoke({"input": "什么是langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'langchain'}`


页面: LangChain
摘要: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

页面: Retrieval-augmented generation
摘要: Retrieval-augmented generation (RAG) is a technique that enables large language models (LLMs) to retrieve and incorporate new information. With RAG, LLMs do not respond to user queries until they refer to a specified set of documents. These documents supplement information from the LLM's pre-existing training data. This allows LLMs to use domain-specific and/or updated information that is not available in the training data. For example, this helps LLM-based chatbots access internal company data or generate respon

{'input': '什么是langchain?',
 'output': 'LangChain 是一个软件框架，旨在帮助将大型语言模型（LLMs）集成到应用程序中。它的主要用途与语言模型的通用用途重叠，包括文档分析与摘要、聊天机器人和代码分析等。\n\n此外，LangChain 可能与检索增强生成（Retrieval-augmented generation, RAG）等技术结合使用。RAG 是一种让大型语言模型能够检索并整合新信息的技术，通过引用指定的文档来补充模型预训练数据中未包含的信息。这种方法可以减少模型的幻觉问题，并提高响应的准确性和透明度。\n\n如果你对 LangChain 的具体功能或应用场景有更多问题，可以进一步探讨！'}

In [52]:
agent_executor.invoke({"input": "我的名字是bob"})



> Entering new AgentExecutor chain...
你好，Bob！很高兴认识你！有什么我可以帮你的吗？

> Finished chain.


{'input': '我的名字是bob', 'output': '你好，Bob！很高兴认识你！有什么我可以帮你的吗？'}

In [53]:
agent_executor.invoke({"input": "我的名字是什么？"})



> Entering new AgentExecutor chain...
我无法知道你的名字，因为这是我们的第一次对话。你可以告诉我你的名字，我会记住并称呼你！

> Finished chain.


{'input': '我的名字是什么？', 'output': '我无法知道你的名字，因为这是我们的第一次对话。你可以告诉我你的名字，我会记住并称呼你！'}

## 3.3 加入对话历史

In [54]:
# 加入用户的对话历史
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [55]:
# 构造agent的处理链
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_tool_messages(x["intermediate_steps"])
) | prompt | model | ToolsAgentOutputParser()

In [59]:
# 加入对话记忆模块
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [60]:
# 封装AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [61]:
agent_executor.invoke({"input": "我的名字是bob"})



> Entering new AgentExecutor chain...
好的，Bob！有什么我可以帮你的吗？或者你只是想让我记住你的名字？ 😏

> Finished chain.


{'input': '我的名字是bob',
 'chat_history': [HumanMessage(content='我的名字是bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='好的，Bob！有什么我可以帮你的吗？或者你只是想让我记住你的名字？ 😏', additional_kwargs={}, response_metadata={})],
 'output': '好的，Bob！有什么我可以帮你的吗？或者你只是想让我记住你的名字？ 😏'}

In [62]:
agent_executor.invoke({"input": "我的名字是什么？"})



> Entering new AgentExecutor chain...
Bob啊Bob，这么快就忘了自己是谁了？需要我帮你写个备忘录贴在脑门上吗？ 😆

> Finished chain.


{'input': '我的名字是什么？',
 'chat_history': [HumanMessage(content='我的名字是bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='好的，Bob！有什么我可以帮你的吗？或者你只是想让我记住你的名字？ 😏', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='我的名字是什么？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Bob啊Bob，这么快就忘了自己是谁了？需要我帮你写个备忘录贴在脑门上吗？ 😆', additional_kwargs={}, response_metadata={})],
 'output': 'Bob啊Bob，这么快就忘了自己是谁了？需要我帮你写个备忘录贴在脑门上吗？ 😆'}

In [63]:
agent_executor.invoke({"input": "圣弗朗西斯科现在的温度是多少？"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`


现在温度是 11.6°C圣弗朗西斯科现在的温度是11.6°C，记得带件外套哦，Bob！ 🌁

> Finished chain.


{'input': '圣弗朗西斯科现在的温度是多少？',
 'chat_history': [HumanMessage(content='我的名字是bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='好的，Bob！有什么我可以帮你的吗？或者你只是想让我记住你的名字？ 😏', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='我的名字是什么？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Bob啊Bob，这么快就忘了自己是谁了？需要我帮你写个备忘录贴在脑门上吗？ 😆', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='圣弗朗西斯科现在的温度是多少？', additional_kwargs={}, response_metadata={}),
  AIMessage(content='圣弗朗西斯科现在的温度是11.6°C，记得带件外套哦，Bob！ 🌁', additional_kwargs={}, response_metadata={})],
 'output': '圣弗朗西斯科现在的温度是11.6°C，记得带件外套哦，Bob！ 🌁'}

# 三、 创建对话机器人

In [64]:
# 自定义的工具（自由发挥）
@tool
def create_your_own(query: str) -> str:
    """可以自定义的功能函数 """
    print(type(query))
    return query[::-1]

In [65]:
# 将之前定义的工具加入工具列表
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [74]:
chat_model = ChatOpenAI(temperature=0, model_name="Qwen/Qwen3-8B", max_tokens=4096,
                        openai_api_key=API_KEY, openai_api_base=BASE_URL, max_retries=3,
                        seed=42, presence_penalty=0.1, frequency_penalty=0.1,
                        extra_body={
                            "enable_thinking": False
                        }
                        )

In [77]:
import panel as pn  # GUI
pn.extension()

import panel as pn
import param

# 定义 cbfs 类
class cbfs(param.Parameterized):
    
    # 初始化函数
    def __init__(self, tools, **params):
        super(cbfs, self).__init__(**params)
        self.panels = []  # 存储 GUI 面板
        self.functions = [convert_to_openai_function(f) for f in tools]  # 将tools格式化为 OpenAI 函数
        self.tools = [{"type": "function", "function": x} for x in self.functions]
        self.model = chat_model.bind(tools=self.tools)  # 创建 ChatOpenAI 模型
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")  # 创建 ConversationBufferMemory
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])  # 创建 ChatPromptTemplate
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_tool_messages(x["intermediate_steps"])
        ) | self.prompt | self.model | ToolsAgentOutputParser()  # 创建处理链
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)  # 创建 AgentExecutor
    
    # 对话链函数
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)

    # 清除历史记录函数
    def clr_history(self, count=0):
        self.panels = []  # 清空存储对话面板的列表
        self.memory.clear()  # 清空对话记忆
        return pn.WidgetBox(scroll=True, height=400, width=539)  # 返回清空后的对话面板


In [80]:
# 创建 cbfs 对象，传递工具列表
cb = cbfs(tools)

# 创建文本输入框组件
inp = pn.widgets.TextInput(placeholder='请输入文本...')

# 创建发送按钮
send_button = pn.widgets.Button(name='发送', button_type='primary')

# 创建清空会话记录按钮
clear_button = pn.widgets.Button(name='清空会话', button_type='danger')

# 初始对话面板
conversation = pn.WidgetBox(scroll=True, height=400, width=539)


# 定义发送消息的函数
def send_message(event):
    query = inp.value
    if query:
        result = cb.convchain(query)
        conversation.objects += result.objects
        inp.value = ''  # 清空输入框


def handle_enter_event(event):
    if inp.value != '':
        send_message(event)


# 监听回车事件
inp.param.watch(handle_enter_event, 'enter_pressed')


# 定义清空会话记录的函数
def clear_conversation(event):
    print(event)
    result = cb.clr_history()
    conversation.objects = result.objects


# 将按钮点击事件与发送消息函数绑定
send_button.on_click(send_message)

# 将按钮点击事件与清空会话记录函数绑定
clear_button.on_click(clear_conversation)

# 创建仪表板，包含标题、输入框、发送按钮和对话记录面板
dashboard = pn.Column(
    pn.pane.Markdown('# 对话机器人'),  # 显示标题
    pn.Row(inp, send_button, clear_button),  # 显示文本输入框和发送按钮
    pn.layout.Divider(),
    pn.panel(conversation, loading_indicator=True),  # 显示对话链面板
    pn.layout.Divider(),
)

dashboard

Column
    [0] Markdown(str)
    [1] Row
        [0] TextInput(placeholder='请输入文本...')
        [1] Button(button_type='primary', name='发送')
        [2] Button(button_type='danger', name='清空会话')
    [2] Divider()
    [3] WidgetBox(height=400, scroll=True, sizing_mode='fixed', width=539)
    [4] Divider()

聊天机器人效果展示如下：

![](../../figures/chat_bot.png)

# 四、英文版提示

总结一下构建对话机器人的流程：

定义tools和functions --> 构建模型 --> 加入中间步骤结果 --> 加入记忆机制 --> 交互界面  

In [81]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [82]:
# 构造agent的处理链
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_tool_messages(x["intermediate_steps"])
) | prompt | model | ToolsAgentOutputParser()

In [83]:
# 加入对话记忆模块
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [84]:
# 封装AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [85]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Well, hello there, Bob! What can I do for you today? Or should I say, what can I do for *you*, the one and only Bob? 😏

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Well, hello there, Bob! What can I do for you today? Or should I say, what can I do for *you*, the one and only Bob? 😏', additional_kwargs={}, response_metadata={})],
 'output': 'Well, hello there, Bob! What can I do for you today? Or should I say, what can I do for *you*, the one and only Bob? 😏'}

In [86]:
agent_executor.invoke({"input": "what is my name"})



> Entering new AgentExecutor chain...
Oh, Bob, Bob, Bob... Are we having a little memory lapse today? Or are you just testing me? Because, *sigh*, your name is **Bob**. Don't make me say it again—unless you want me to. 😏

> Finished chain.


{'input': 'what is my name',
 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Well, hello there, Bob! What can I do for you today? Or should I say, what can I do for *you*, the one and only Bob? 😏', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Oh, Bob, Bob, Bob... Are we having a little memory lapse today? Or are you just testing me? Because, *sigh*, your name is **Bob**. Don't make me say it again—unless you want me to. 😏", additional_kwargs={}, response_metadata={})],
 'output': "Oh, Bob, Bob, Bob... Are we having a little memory lapse today? Or are you just testing me? Because, *sigh*, your name is **Bob**. Don't make me say it again—unless you want me to. 😏"}

In [87]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`


现在温度是 13.0°CThe current temperature in San Francisco is a crisp **13.0°C**. Perfect weather for a hoodie and some sass, don't you think, Bob? 😏

> Finished chain.


{'input': 'whats the weather in sf?',
 'chat_history': [HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Well, hello there, Bob! What can I do for you today? Or should I say, what can I do for *you*, the one and only Bob? 😏', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='what is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Oh, Bob, Bob, Bob... Are we having a little memory lapse today? Or are you just testing me? Because, *sigh*, your name is **Bob**. Don't make me say it again—unless you want me to. 😏", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="The current temperature in San Francisco is a crisp **13.0°C**. Perfect weather for a hoodie and some sass, don't you think, Bob? 😏", additional_kwargs={}, response_metadata={})],
 'output': "The current temperature in